In [438]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import datetime
import numpy as np
from sqlalchemy import create_engine

# SQLAlchemy connectable
cnx = create_engine('sqlite:///db-carenewable.db').connect()

# table named 'CAISO_ALL_SUPPLY_DATA' will be returned as a dataframe.
supply_df = pd.read_sql_table('CAISO-ALL-SUPPLY-DATA', cnx)

In [439]:
# Create sum column for each row supply (MW)
supply_df['Sum'] = supply_df['Renewables'] + supply_df['Largehydro'] + supply_df['Imports'] + supply_df['Batteries'] + supply_df['Nuclear'] + supply_df['Coal'] + supply_df['Other'] + supply_df['Naturalgas']
print(supply_df.dtypes)
supply_df.head()

Renewables       float64
Naturalgas       float64
Largehydro       float64
Imports          float64
Batteries        float64
Nuclear          float64
Coal             float64
Other            float64
Date              object
Time              object
DateTime          object
UnixTimestamp      int64
Sum              float64
dtype: object


,Renewables,Naturalgas,Largehydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,UnixTimestamp,Sum
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,4/10/2018,0:00:00,4/10/2018 0:00,1523318400,22879.0
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,4/10/2018,0:05:00,4/10/2018 0:05,1523318700,22794.0
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,4/10/2018,0:10:00,4/10/2018 0:10,1523319000,22743.0
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,4/10/2018,0:15:00,4/10/2018 0:15,1523319300,22488.0
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,4/10/2018,0:20:00,4/10/2018 0:20,1523319600,22368.0


In [440]:
baby_df = supply_df['Time'].str.split(":", expand=True)
print(baby_df)

         0   1   2
0        0  00  00
1        0  05  00
2        0  10  00
3        0  15  00
4        0  20  00
...     ..  ..  ..
412951  23  35  00
412952  23  40  00
412953  23  45  00
412954  23  50  00
412955  23  55  00

[412956 rows x 3 columns]


In [441]:
baby_df[0] = baby_df[0].astype(int)
baby_df[1] = baby_df[1].astype(int)

In [442]:
baby_df= baby_df.drop([2],1)

C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [443]:
baby_df.dtypes

0    int32
1    int32
dtype: object

In [444]:
# Rename columns and create years value for calculating demand_EV
baby_df = baby_df.rename(columns={baby_df.columns[0]: 'Hours'})
baby_df = baby_df.rename(columns={baby_df.columns[1]: 'Minutes'})

In [445]:
supply_df['Date'] = pd.to_datetime(supply_df['Date'])

#Create a new column for'Year'
baby_df['Year'] = supply_df['Date'].dt.year
#Convert to integers to be used in computations
baby_df['Year'] = baby_df['Year'].astype(int)

baby_df.head(100)

,Hours,Minutes,Year
0,0,0,2018
1,0,5,2018
2,0,10,2018
3,0,15,2018
4,0,20,2018
...,...,...,...
95,7,55,2018
96,8,0,2018
97,8,5,2018
98,8,10,2018


In [446]:
#Make a date string column 
date_df = pd.DataFrame()
supply_df['Date1']=supply_df['Date'].astype(str)

date_df = supply_df['Date1'].str.split("-", expand=True)
date_df['DateAttribute'] = date_df[1] + date_df[2]
date_df = date_df.drop([0,1,2],1)


C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


In [447]:
baby_df['DemandMW'] = 0.0

mask = (baby_df['Year'] == 2018) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask1 = (baby_df['Year'] == 2019) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask2 = (baby_df['Year'] == 2020) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)


baby_df.loc[mask,'DemandMW'] = 6.2356
baby_df.loc[mask1,'DemandMW'] = 8.2818
baby_df.loc[mask2,'DemandMW'] = 9.9174

In [448]:
calc_df = pd.merge(baby_df, date_df, left_index=True, right_index=True, how='outer')
calc_df['DateTime'] = supply_df['DateTime']
print(calc_df.dtypes)
calc_df

Hours              int32
Minutes            int32
Year               int32
DemandMW         float64
DateAttribute     object
DateTime          object
dtype: object


,Hours,Minutes,Year,DemandMW,DateAttribute,DateTime
0,0,0,2018,6.2356,0410,4/10/2018 0:00
1,0,5,2018,6.2356,0410,4/10/2018 0:05
2,0,10,2018,6.2356,0410,4/10/2018 0:10
3,0,15,2018,6.2356,0410,4/10/2018 0:15
4,0,20,2018,6.2356,0410,4/10/2018 0:20
...,...,...,...,...,...,...
412951,23,35,2022,9.9174,0324,3/24/2022 23:35
412952,23,40,2022,9.9174,0324,3/24/2022 23:40
412953,23,45,2022,9.9174,0324,3/24/2022 23:45
412954,23,50,2022,9.9174,0324,3/24/2022 23:50


In [449]:
final_df = pd.merge(supply_df, calc_df, left_index=True, right_index=True, how='outer')
final_df['DateTime'] = final_df['DateTime_y']
final_df = final_df.drop(['Date1','DateTime_x','DateTime_y'],1)
final_df = final_df[final_df['Year'] <= 2020]
print(final_df.shape)
print(final_df.dtypes)
final_df

C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


(285662, 18)
Renewables              float64
Naturalgas              float64
Largehydro              float64
Imports                 float64
Batteries               float64
Nuclear                 float64
Coal                    float64
Other                   float64
Date             datetime64[ns]
Time                     object
UnixTimestamp             int64
Sum                     float64
Hours                     int32
Minutes                   int32
Year                      int32
DemandMW                float64
DateAttribute            object
DateTime                 object
dtype: object


,Renewables,Naturalgas,Largehydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,UnixTimestamp,Sum,Hours,Minutes,Year,DemandMW,DateAttribute,DateTime
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,2018-04-10,0:00:00,1523318400,22879.0,0,0,2018,6.2356,0410,4/10/2018 0:00
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,2018-04-10,0:05:00,1523318700,22794.0,0,5,2018,6.2356,0410,4/10/2018 0:05
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,2018-04-10,0:10:00,1523319000,22743.0,0,10,2018,6.2356,0410,4/10/2018 0:10
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,2018-04-10,0:15:00,1523319300,22488.0,0,15,2018,6.2356,0410,4/10/2018 0:15
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,2018-04-10,0:20:00,1523319600,22368.0,0,20,2018,6.2356,0410,4/10/2018 0:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285657,3183.0,8540.0,764.0,8800.0,122.0,1144.0,12.0,0.0,2020-12-31,23:35:00,1609457700,22565.0,23,35,2020,9.9174,1231,12/31/2020 23:35
285658,3294.0,8450.0,718.0,8804.0,99.0,1144.0,12.0,0.0,2020-12-31,23:40:00,1609458000,22521.0,23,40,2020,9.9174,1231,12/31/2020 23:40
285659,3410.0,8300.0,731.0,8725.0,90.0,1143.0,13.0,0.0,2020-12-31,23:45:00,1609458300,22412.0,23,45,2020,9.9174,1231,12/31/2020 23:45
285660,3456.0,8217.0,753.0,8621.0,91.0,1144.0,12.0,0.0,2020-12-31,23:50:00,1609458600,22294.0,23,50,2020,9.9174,1231,12/31/2020 23:50


In [450]:
df1= pd.DataFrame()
df1 = final_df.groupby(['DateAttribute','Time'])['Sum'].agg('mean')
df1 = df1.reset_index()
df1 = df1.rename(columns={df1.columns[2]: 'AvgSUM(3yr)'}).round(2)

In [451]:
# Create 2020 avg Dataframe
avg_df_2020 = df1
avg_df_2020['Year'] = 2020
print(avg_df_2020.shape)

# Create 2018 and 2019 avg DataFrames: filter out Feb 29th and missing dates for 2018 before Apr 10th
avg_df_2019 = avg_df_2020[avg_df_2020['DateAttribute'] != "0229"]
avg_df_2019['Year'] = 2019
print(avg_df_2019.shape)

avg_df_2018 = avg_df_2019
avg_df_2018['Year'] = 2018
avg_df_2018['DateAttribute'] = avg_df_2018['DateAttribute'].astype(int)
avg_df_2018 = avg_df_2018[avg_df_2018['DateAttribute'] >= 410]
print(avg_df_2018.shape)

(105408, 4)
(105120, 4)
(76608, 4)


C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [452]:
# Create list of avg DataFrames
avg_dfs_list = [avg_df_2020, avg_df_2019, avg_df_2018]

# Concat list
avg_df = pd.concat(avg_dfs_list)
print(avg_df.shape)
print(avg_df.dtypes)
avg_df.head()

(287136, 4)
DateAttribute     object
Time              object
AvgSUM(3yr)      float64
Year               int64
dtype: object


,DateAttribute,Time,AvgSUM(3yr),Year
0,0101,0:00:00,22735.5,2020
1,0101,0:05:00,22789.0,2020
2,0101,0:10:00,22778.0,2020
3,0101,0:15:00,22732.0,2020
4,0101,0:20:00,22691.5,2020


In [453]:
# Create 'DateTime' column for avg_df
avg_df['Date'] = avg_df['DateAttribute'].astype(str) + avg_df['Year'].astype(str)

val_list = avg_df['Date'].to_list()

date_list = []

for i in val_list:
    date_list.append(i[:2] + '/' + i[2:4] + '/' + i[4:])

avg_df['Date'] = date_list

avg_df['DateTime'] = avg_df['Date'].astype(str) + "T" + avg_df['Time'].astype(str)

avg_df['DateTime'] = pd.to_datetime(avg_df['DateTime'], errors = 'coerce')
avg_df['Date'] = pd.to_datetime(avg_df['Date'], errors = 'coerce')

print(avg_df.shape)
print(avg_df.dtypes)
avg_df.head()

(287136, 6)
DateAttribute            object
Time                     object
AvgSUM(3yr)             float64
Year                      int64
Date             datetime64[ns]
DateTime         datetime64[ns]
dtype: object


,DateAttribute,Time,AvgSUM(3yr),Year,Date,DateTime
0,0101,0:00:00,22735.5,2020,2020-01-01,2020-01-01 00:00:00
1,0101,0:05:00,22789.0,2020,2020-01-01,2020-01-01 00:05:00
2,0101,0:10:00,22778.0,2020,2020-01-01,2020-01-01 00:10:00
3,0101,0:15:00,22732.0,2020,2020-01-01,2020-01-01 00:15:00
4,0101,0:20:00,22691.5,2020,2020-01-01,2020-01-01 00:20:00


In [454]:
final_df_18_20 = final_df
final_df_18_20['Date'] = pd.to_datetime(final_df_18_20['Date'], errors = 'coerce')
final_df_18_20['DateTime'] = pd.to_datetime(final_df_18_20['DateTime'], errors = 'coerce')
print(f"The shape of final_df_18_20 is {final_df_18_20.shape}")
print(final_df_18_20.dtypes)
final_df_18_20.head()

The shape of final_df_18_20 is (285662, 18)
Renewables              float64
Naturalgas              float64
Largehydro              float64
Imports                 float64
Batteries               float64
Nuclear                 float64
Coal                    float64
Other                   float64
Date             datetime64[ns]
Time                     object
UnixTimestamp             int64
Sum                     float64
Hours                     int32
Minutes                   int32
Year                      int32
DemandMW                float64
DateAttribute            object
DateTime         datetime64[ns]
dtype: object


,Renewables,Naturalgas,Largehydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,UnixTimestamp,Sum,Hours,Minutes,Year,DemandMW,DateAttribute,DateTime
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,2018-04-10,0:00:00,1523318400,22879.0,0,0,2018,6.2356,0410,2018-04-10 00:00:00
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,2018-04-10,0:05:00,1523318700,22794.0,0,5,2018,6.2356,0410,2018-04-10 00:05:00
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,2018-04-10,0:10:00,1523319000,22743.0,0,10,2018,6.2356,0410,2018-04-10 00:10:00
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,2018-04-10,0:15:00,1523319300,22488.0,0,15,2018,6.2356,0410,2018-04-10 00:15:00
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,2018-04-10,0:20:00,1523319600,22368.0,0,20,2018,6.2356,0410,2018-04-10 00:20:00


In [465]:
avg_datetime = avg_df['DateTime'].to_list()
final_datetime = final_df_18_20['DateTime'].to_list()

In [ ]:
# Get elements which are in first_list but not in sec_list
diff1 = []
for elem in avg_datetime:
    if elem not in final_datetime:
        diff1.append(elem)
#print(diff1)

In [ ]:
print(diff1)